# Converting a polygon map to hexagons

## About

- This script demonstrates the usage of hexagonizing function to convert a
polygon map into hexagons. Additionally, the hexagons can be converted into
center points.
- Use case: the EUNIS map information needs to be translated to match the hexagons of the BBT analysis.

__author__ = "Willem Boone" <br>
__email__ = "willem.boone@vliz.be"

## Imports

In [32]:
import geopandas as gpd
import os
from hexagonize import hexagonize_dominant
from hexagonize import hexagonize_keep_all
from hexagonize import hexagonize_density
from hexagonize import to_centroids
from pathlib import Path
from pprint import pprint

## Settings
Add the path to following files:
- map: the map you want to convert
- hexagons: shapefile containing hexagons you need to convert the map to.
- processing: a directory where to store results

Notice: make sure the map and hexagon file have the same projection.

In [33]:
map = "EUSeaMap_2023.gdb"
hexagons = "hexagon_grid/hexagon_grid.shp"
out_path = "output_map_conversion"

Deduct map name and check output path.

In [34]:
map_name = Path(map).stem
if not os.path.exists(out_path):
    os.makedirs(out_path)

## Read data

In [35]:
hexagon_gdf = gpd.read_file(hexagons)
map_gdf = gpd.read_file(map, mask=hexagon_gdf)
map_gdf.to_file(os.path.join(out_path, f"clip_{map_name}.shp"))

C:\Users\willem.boone\AppData\Local\Temp\ipykernel_21024\3580200146.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  map_gdf.to_file(os.path.join(out_path, f"clip_{map_name}.shp"))


Inspect data

In [36]:
print(map_gdf.columns)

Index(['ModelCode', 'Oxygen', 'Salinity', 'Energy', 'Biozone', 'Substrate',
       'EUNIScomb', 'EUNIScombD', 'Allcomb', 'AllcombD', 'SalcombD',
       'MSFD_BBHT', 'EUNIS2019C', 'EUNIS2019D', 'All2019D', 'All2019DL2',
       'RegionalD', 'Val_comm', 'Shape_Length', 'Shape_Area', 'geometry'],
      dtype='object')


In [37]:
print(hexagon_gdf.columns)

Index(['FID', 'geometry'], dtype='object')


The map will be clipped to extend of the hexagons, the clipped data can be saved:

In [38]:
map_gdf.to_file(os.path.join(out_path, f"clip_{map_name}.shp"))

C:\Users\willem.boone\AppData\Local\Temp\ipykernel_21024\1286735209.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  map_gdf.to_file(os.path.join(out_path, f"clip_{map_name}.shp"))


## Processing

Convert your map to hexagons.

#### Method 1: dominance [hexagonize_dominant]
Convert a spatial map into a map of hexagons: an overlay is made between the original map and the hexagons. In this overlay, several classes of the original map can occur within a single hexagon. The dominant class in terms of area within this hexagon (i.e. the class with the largest presence) will be assigned to the resulting hexagon map.

#### Method 2: keep all & duplicate [hexagonize_keep_all]
Convert a spatial map into a map of hexagons: a spatial join is made between the original map and the hexagons. When several classes of the original map occur within a single hexagon, this will result in multiple records having the same geometry but holding different attributes from the original map.

#### Method 3: keep all & duplicate [hexagonize_density]
Convert a spatial map into a map of hexagons: Convert a spatial map into a map of hexagons: a spatial overlay is made between the original map and the hexagons. When several classes of the original map occur within a single hexagon, this will result in multiple records having the same geometry but holding different attributes from the original map. Furthermore, a column density contains the proportion of the original class in a particular hexagon. For area computations, all data will be reprojected to a projected crs default 32631. The output will be delivere once again in the hexagon crs.


In [39]:
map_processed = hexagonize_density(map_gdf=map_gdf, hexagon_gdf=hexagon_gdf)

C:\Users\willem.boone\Documents\projects\MARBEFES\EVA\test\pythonProject\hexagonize.py:69: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hexagon_gdf["hexagon_area"] = hexagon_gdf.area
C:\Users\willem.boone\Documents\projects\MARBEFES\EVA\test\pythonProject\hexagonize.py:77: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlay["v_map_area"] = overlay.area


In [40]:
print(map_processed.columns)

Index(['FID', 'hexagon_area', 'geometry', 'ModelCode', 'Oxygen', 'Salinity',
       'Energy', 'Biozone', 'Substrate', 'EUNIScomb', 'EUNIScombD', 'Allcomb',
       'AllcombD', 'SalcombD', 'MSFD_BBHT', 'EUNIS2019C', 'EUNIS2019D',
       'All2019D', 'All2019DL2', 'RegionalD', 'Val_comm', 'Shape_Length',
       'Shape_Area', 'Density'],
      dtype='object')


In [41]:
print(type(map_processed))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [42]:
for item in map_processed.head(100).itertuples():
    print(f"Hexagong {item.FID} contains {round(item.Density*100, 2)}% of MSFD_BBHT: {item.MSFD_BBHT}")

Hexagong 0 contains 0.11% of MSFD_BBHT: Infralittoral sand
Hexagong 0 contains 0.11% of MSFD_BBHT: Offshore circalittoral sand
Hexagong 0 contains 0.11% of MSFD_BBHT: Offshore circalittoral sand
Hexagong 0 contains 0.0% of MSFD_BBHT: Offshore circalittoral sand
Hexagong 0 contains 0.11% of MSFD_BBHT: Circalittoral sand
Hexagong 0 contains 16.21% of MSFD_BBHT: Circalittoral sand
Hexagong 0 contains 5.43% of MSFD_BBHT: Offshore circalittoral sand
Hexagong 0 contains 0.07% of MSFD_BBHT: Offshore circalittoral sand
Hexagong 0 contains 2.08% of MSFD_BBHT: Circalittoral coarse sediment
Hexagong 0 contains 0.01% of MSFD_BBHT: Circalittoral coarse sediment
Hexagong 0 contains 75.78% of MSFD_BBHT: Offshore circalittoral coarse sediment
Hexagong 1 contains 1.16% of MSFD_BBHT: Infralittoral sand
Hexagong 1 contains 4.18% of MSFD_BBHT: Infralittoral sand
Hexagong 1 contains 0.0% of MSFD_BBHT: Infralittoral sand
Hexagong 1 contains 5.68% of MSFD_BBHT: Circalittoral sand
Hexagong 1 contains 4.36% of

Since the EVA analysis uses points, the hexagons can be converted to their centroid geometry.

In [43]:
map_processed = to_centroids(map_processed)

C:\Users\willem.boone\Documents\projects\MARBEFES\EVA\test\pythonProject\hexagonize.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf.geometry.centroid


In [44]:
print(type(map_processed))

<class 'geopandas.geodataframe.GeoDataFrame'>


## Post processing

EVA analysis requires some additional fields which will be added in the code below.

In [45]:
map_processed["EventDate"] = ""
map_processed["FieldNumber"] = ""
map_processed["ScientificName"] = map_name

## Save results

In [46]:
dest = os.path.join(out_path, f"{map_name}_hexagon_centroids.csv")
map_processed.to_csv(dest)
dest = os.path.join(out_path, f"{map_name}_hexagon_centroids.shp")
map_processed.to_file(dest)

C:\Users\willem.boone\AppData\Local\Temp\ipykernel_21024\191313916.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  map_processed.to_file(dest)
